In [24]:
!wget -q --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1p1wjaqpTh_5RHfJu4vUh8JJCdKwYMHCp' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1p1wjaqpTh_5RHfJu4vUh8JJCdKwYMHCp" -O data && rm -rf /tmp/cookies.txt
!unzip -q data && rm data

In [30]:
!ls lfw2/lfw2/Aaron_Eckhart/ | wc -l

5749


In [40]:
import glob

files = glob.glob('lfw2/lfw2/George_W_Bush/*.jpg')
len(files)

530

In [38]:
files

['lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0001.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0002.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0003.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0004.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0005.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0006.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0007.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0008.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0009.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0010.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0011.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0012.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0013.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0014.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0015.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0016.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Alejandro_Toledo_0017.jpg',
 'lfw2/lfw2/Alejandro_Toledo/Al